In [31]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import random
from models import create_mscnn_model
import pickle as pk
# https://www.mdpi.com/1424-8220/20/1/166

## Model search

In [32]:
# Data generator 
class PRONOSTIASequence(tf.keras.utils.Sequence):

    def __init__(self, data, batches_per_epoch=1000, batch_size=32, split_channel=False):
        self.data = data
        self.batch_size = batch_size
        self.batches_per_epoch = batches_per_epoch
        self.bearings = self.data[['Condition', 'Bearing']].drop_duplicates().values
        self.data = {}
        self.rul_max = {}
        self.split_channel = split_channel
        D = data
        for cond, bearing in self.bearings:
            d = D[(D.Condition == cond) & (D.Bearing==bearing)]
            self.rul_max[(cond, bearing)] = d.RUL.max()
            self.data[(cond, bearing)] = d[['V_acc', 'H_acc', 'RUL']].values
    def __len__(self):
        return self.batches_per_epoch

    def __getitem__(self, idx):
        D = self.data
        
        if self.split_channel:
            X = np.zeros(shape=(self.batch_size, 2, 256//2, 2))
            Y = np.zeros(shape=(self.batch_size,))
            for i in range(self.batch_size):
                cond, bearing = self.bearings[random.randint(0, self.bearings.shape[0]-1)]
                Db = self.data[(cond, bearing)]
                L = (Db.shape[0] // 256) 

                k = random.randint(0, L-2) * 256

                l = 256//2
                X[i, :, :, 0] = Db[k:k+l, :2].T
                X[i, :, :, 1] = Db[k+l:k+2*l, :2].T
                Y[i] = Db[k+1:k+2*l, 2][-1] / self.rul_max[(cond, bearing)]   
        else:
            X = np.zeros(shape=(self.batch_size, 2, 256, 1))
            Y = np.zeros(shape=(self.batch_size,))
            for i in range(self.batch_size):
                cond, bearing = self.bearings[random.randint(0, self.bearings.shape[0]-1)]
                Db = self.data[(cond, bearing)]
                L = (Db.shape[0] // 256) 
                k = random.randint(0, L-2) * 256
                X[i, :, :, 0] = Db[k:k+256, :2].T
                Y[i] = Db[k:k+256, 2][-1] / self.rul_max[(cond, bearing)]  
        return X, Y
        

In [ ]:
from ray import tune
import ray

ray.shutdown()
ray.init(num_cpus=4, num_gpus=2)



def phm21_ccn(config):
     

    X_train = pd.read_csv('/home/dasolma/papers/xai/data/pronostia_train.csv')
    X_test = pd.read_csv('/home/dasolma/papers/xai/data/pronostia_test.csv')
    X = pd.concat((X_train,X_test), axis=0)
    
    
    X.loc[:, 'V_acc'] = X.V_acc / 50
    X.loc[:, 'H_acc'] = X.H_acc / 50

    X_test = X[X.Bearing.isin([1,3,4,7])]
    X_train = X[X.Bearing.isin([2,5,6])]

    gen_train = PRONOSTIASequence(X_train)
    gen_val = PRONOSTIASequence(X_test, batches_per_epoch=5000)
    
    epochs = config.pop("epochs")
    
    base_model = create_mscnn_model((2,128,2),**config)

    def pronostia_lambda_layer(x):
        return tf.keras.backend.concatenate([x[:,:, 128:,:], 
                                             x[:,:, :128,:]], axis=-1)

    raw_input = tf.keras.layers.Input((2, 256, 1))
    x = tf.keras.layers.Lambda(pronostia_lambda_layer)(raw_input)
    x = base_model(x)

    m = tf.keras.models.Model(raw_input, x)
    
    from scoring import NASAScore, PHM21Score
    m.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(lr=config['lr']), 
        metrics=[NASAScore(), PHM21Score(), tf.keras.metrics.MeanAbsoluteError(name="MAE")])
 

    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
    rlr = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
    history = m.fit(gen_train, validation_data=gen_val,
                    batch_size=32, epochs=epochs, verbose=0,
                   callbacks=[es, rlr])
    history = history.history
    tune.report(score=history['val_loss'][-1])
    

from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.schedulers import ASHAScheduler
space = {
    "block_size": (1.51, 4.5),
    "msblocks": (-0.51, 4.5),
    "nblocks": (1.51, 4.5),
    "l1": (0, 1e-3),
    "l2": (0, 1e-3),
    "dropout": (0, 0.9),
    "lr": (1e-5, 1e-3),
    "fc1": (64, 256),
    "conv_activation": (-0.51, 2.5),
    "dense_activation": (-0.51, 2.5),
    "dilation_rate": (0.51, 10.49),
    "kernel_size": (-0.51, 1.5),
    "f1": (2.51, 15.5),
    "f2": (2.51, 15.5),
    "f3": (2.51, 15.5),
}


bayesopt = BayesOptSearch(space=space, mode="min", metric="score")
scheduler=ASHAScheduler(metric="score", mode="min", max_t=3600, time_attr='training_iteration')

analysis = tune.run(
    phm21_ccn,
    config={
        "epochs": 100,
        "preprocess_layer": "pronostia",
    },
    resources_per_trial={'gpu': 1},
    num_samples=30,
    search_alg=bayesopt,
    log_to_file=False,
    scheduler=scheduler
    )


In [ ]:
#pk.dump(analysis._checkpoints, open('tune_checkpoint_pronostia', 'wb'))

### Train best model

In [30]:
checkpoints = pk.load(open('tune_checkpoint_pronostia.pk', 'rb'))
config = sorted(checkpoints, key=lambda c: c['last_result']['score'])[0]['config']
config

{'epochs': 100,
 'block_size': 1.7745925811352392,
 'conv_activation': 0.079908415881627,
 'dense_activation': -0.37386586037928043,
 'dilation_rate': 3.756796701017378,
 'dropout': 0.3498095607205338,
 'f1': 6.034823922742907,
 'f2': 13.275300243883562,
 'f3': 7.144225713749725,
 'fc1': 117.9394258599771,
 'kernel_size': 0.5808191271480794,
 'l1': 0.00014092422497476265,
 'l2': 0.0008021969807540396,
 'lr': 8.380513724297312e-05,
 'nblocks': 4.460791940435547}

### Prepare data

In [12]:
X_train = pd.read_csv('/home/dasolma/papers/xai/data/pronostia_train.csv')
X_test = pd.read_csv('/home/dasolma/papers/xai/data/pronostia_test.csv')
X = pd.concat((X_train,X_test), axis=0)


X.loc[:, 'V_acc'] = X.V_acc #/ 50
X.loc[:, 'H_acc'] = X.H_acc #/ 50

X_test = X[X.Bearing.isin([1,3,4,7])]
X_train = X[X.Bearing.isin([2,5,6])]



In [5]:
gen_train = PRONOSTIASequence(X_train, split_channel=False)
gen_val = PRONOSTIASequence(X_test, batches_per_epoch=625, batch_size=256, split_channel=False)
gen_train.__getitem__(0)[0].shape

(32, 2, 256, 1)

In [6]:
epochs = config.pop('epochs')

In [9]:
config['pooling_kernel'] = (1, 2)
config['msblocks'] = 0
config['kernel_size'] = 1
config['preprocess_layer'] = 'pronostia'
m = create_mscnn_model((2, 256, 1),**config)

from scoring import *
m.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(lr=config['lr']), 
        metrics=[NASAScore(), PHM21Score(), tf.keras.metrics.MeanAbsoluteError(name="MAE")])
    
    
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
rlr = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
history = m.fit(gen_train, validation_data=gen_val,
                batch_size=32, epochs=epochs, verbose=1,
               callbacks=[es, rlr])

Epoch 1/100
1000/1000 [==============================] - 39s 39ms/step - loss: 12.2850 - NASA_score: 0.0312 - Score: 0.2215 - MAE: 0.3244 - val_loss: 10.9125 - val_NASA_score: 0.0309 - val_Score: 0.1967 - val_MAE: 0.3077 - lr: 8.3805e-05
Epoch 2/100
1000/1000 [==============================] - 38s 38ms/step - loss: 9.4260 - NASA_score: 0.0203 - Score: 0.1460 - MAE: 0.2231 - val_loss: 7.8858 - val_NASA_score: 0.0210 - val_Score: 0.1409 - val_MAE: 0.2186 - lr: 8.3805e-05
Epoch 3/100
1000/1000 [==============================] - 38s 38ms/step - loss: 6.2708 - NASA_score: 0.0181 - Score: 0.1313 - MAE: 0.2009 - val_loss: 4.7088 - val_NASA_score: 0.0209 - val_Score: 0.1385 - val_MAE: 0.2168 - lr: 8.3805e-05
Epoch 4/100
1000/1000 [==============================] - 39s 39ms/step - loss: 3.3356 - NASA_score: 0.0173 - Score: 0.1264 - MAE: 0.1924 - val_loss: 2.1561 - val_NASA_score: 0.0198 - val_Score: 0.1323 - val_MAE: 0.2071 - lr: 8.3805e-05
Epoch 5/100
1000/1000 [==============================]

In [10]:
#m.save('../data/models/pronostia/model.h5')

In [33]:
from scoring import *
from models import SplitTS
m = tf.keras.models.load_model('../data/models/pronostia/model.h5', 
                                   custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU,
                                                  'NASAScore': NASAScore,
                                                  'SplitTS': SplitTS,
                                                  'PHM21Score': PHM21Score})

In [35]:
m.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2, 256, 1)]       0         
_________________________________________________________________
split_ts_1 (SplitTS)         (None, 2, 128, 2)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 128, 64)        1344      
_________________________________________________________________
batch_normalization_8 (Batch (None, 2, 128, 64)        256       
_________________________________________________________________
activation_10 (Activation)   (None, 2, 128, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 128, 64)        41024     
_________________________________________________________________
batch_normalization_9 (Batch (None, 2, 128, 64)        256 

### Select samples for XAI methods validation

In [16]:
X, Y = gen_val.__getitem__(0)

In [17]:
pk.dump(X, open("../data/models/pronostia/samples.pk", "wb"))

In [18]:
pk.dump(Y, open("../data/models/pronostia/targets.pk", "wb"))